# 3D7 Finite element method - laboratory notebook

---


## Create a copy before you begin


1. Click **Copy to Drive** using the icon on the top to create a copy of this notebook in your Cambridge Google Drive.

<img src='https://docs.google.com/uc?export=download&id=1c5aWac_AK4AUA9KvlROIJS9vca0UJUfL' height=150/>

2. Click **File > Locate in Drive** to locate your own copy. Rename the notebook appropriately.
3. Do not work directly in this original notebook as you will not be able to save in this original notebook.



## Introduction to Google Colab and Jupyter Notebook

We will use [Jupyter Notebook](https://jupyter.org/) hosted by Google Colab for interactive coding. To use this notebook effectively, you must know the following:

1. Jupyter Notebook has two types of cell i.e. *text* cell and *code* cell. You can typeset and format a *text* cell to provide information about your codes. *Code* cells are executable using the built-in Python interpreter. Each *code* cell has a "Play" icon on its left. To execute a *code* cell, either click the "Play" button or press "Shift" + "Enter" with your keyboard.
2. Google Colab has a built-in Python interpreter and runs on Ubuntu OS. Any Python packages that you install here stay in the cloud. Each time you close and reopen this Notebook, you have to run again all *code* cells including the cells that install the Python packages.
3. Always run *code* cells sequentially i.e. from top to bottom. After you execute successfully a *code* cell, a green tick and a number e.g. [1] or [2] indicating the execution sequence will appear on its left.
4. Save your notebook by clickling **File > Save** or using the keyboard shortcut "Ctrl" + "s".
5. After running a *code* cell, if you wish to clear the output, you can click the cell, then hover your mouse to the top right of the cell and find **More cell actions > Clear output**. Clearing the output helps keep your notebook organised especially for *code* cells that install Python packages.
6. If you wish to clear all Python variables so that the notebook returns to the initial state, click **Runtime > Factory reset runtime**. This should save you time from closing and re-opening your notebook.
7. You can always re-create a copy from the original notebook should you run into bugs that you cannot fix.
8. There is a table of contents on the left to help you navigate the notebook.

More information about Google Colab can be found in [Google Colab's Frequently Asked Questions](https://research.google.com/colaboratory/faq.html).

## Summary of tasks

The following summarises the tasks that you should undertake for your lab investigation. These tasks are designed to help you understand and should not be treated as a guide to your report. Please refer to the guide in the lab handout for writing your report.

*   **TASK 1**: Investigate the effect of mesh refinement on the convergence of the finite element result.
*   **TASK 2**: Investigate how to implement the weak form if the prescribed traction is changed to prescribed displacement on the rightmost edge of the strip.
*   **TASK 3**: State two factors that can increase the number of degrees of freedom i.e. the size of the assembled finite element linear system.
*   **TASK 4**: Examine the relationship between the magnitude of the applied traction and the resulting stress field.

Each task will be further elaborated at the appropriate locations of this notebook. As you work through the notebook, watch for the tags **TASK # guide** and read the elaborated description.

# Step 1. Install [FEniCS](https://fenicsproject.org/)

FEniCS is an open-source finite element package with a Python interface. Run the following cell to install FEniCS on Google Colab:

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# Step 2. Install [Gmsh](https://gmsh.info/)

Gmsh is an open source finite element mesh generator with built-in computer-aided design (CAD) engine. You can create a geometry and subsequently mesh the geometry. Gmsh has a grahics user interface and a Python interface. In this notebook, we will be using Gmsh's Python interface.

In [ ]:
!pip3 install gmsh==4.7.0
import sys
sys.path += ['/usr/local/lib/python3.7/site-packages/gmsh-4.7.0-Linux64-sdk/lib']

# Step 3. Create and mesh the geometry

We first create the geometry. The geometry is a quarter of a strip in $\mathbb{R}^2$, see Figure 1 in the labaratory handout for the geometric parameters. Subsequently, we use Gmsh's built-in mesher to help us mesh the geometry.

In [ ]:
# Import some packages: gmsh, the sys package to allow command line commands to be executed from the Python interpreter and the math package
import gmsh
import sys
import math

# Initialise gmsh
gmsh.initialize(sys.argv)
# Ask gmsh to display information in the terminal
gmsh.option.setNumber("General.Terminal", 1)
# Select the gmsh file format
gmsh.option.setNumber("Mesh.MshFileVersion", 2.0)

# Create a model and name it "PlateSymmetric"
model = gmsh.model
model.add("PlateSymmetric")

# Define the parameters
L = 150;  # half-length of the plate
w = 50;   # half-width of the plate
r = 14.8; # radius of the hole
hmin = 8; # min element size
hmax = 8; # max element size

# Create Points to define the mesh geometry
holecenter        = model.geo.addPoint(0, 0, 0, hmin, 1)
holeedge1         = model.geo.addPoint(r, 0, 0, hmin, 2)
bottomrightcorner = model.geo.addPoint(L, 0, 0, hmax, 3)
toprightcorner    = model.geo.addPoint(L, w, 0, hmax, 4)
topleftcorner     = model.geo.addPoint(0, w, 0, hmax, 5)
holeedge2         = model.geo.addPoint(0, r, 0, hmin, 6)

# Create straight lines
lines = []
lines.append(model.geo.addLine(2, 3, 1))
lines.append(model.geo.addLine(3, 4, 2))
lines.append(model.geo.addLine(4, 5, 3))
lines.append(model.geo.addLine(5, 6, 4))

# Create a circle arc
lines.append(model.geo.addCircleArc(6, 1, 2))
# Define a curve loop by combining the lines and the circle arc
curveloop = model.geo.addCurveLoop([1,2,3,4,5])
# Define a surface inside the curve loop
disk = model.geo.addPlaneSurface([curveloop])

# Synchronize the CAD kernel with the gmsh model
gmsh.model.geo.synchronize()
# Create the 2D mesh
model.mesh.generate(2)
# Write the mesh in a .msh file
gmsh.write("mesh.msh")
# Finalize gmsh
gmsh.finalize()

> **TASK 1 guide**: Notice the parameters *hmin* and *hmax* in the cell above. Adjust these parameters to investigate the effect of coarsening/ refining the finite element mesh.

After meshing the geometry it is important to visualise the mesh. FEniCS provides the utility to visualise the mesh. To this end, we convert the Gmsh file format (.msh) into FEniCS file format (.xml) using *meshconvert*.

In [ ]:
from fenics import *
from dolfin_utils.meshconvert import meshconvert
meshconvert.convert2xml("mesh.msh", "mesh2.xml", "gmsh")

Eventually, we read the .xml file within FEniCS to visualise the mesh.

In [ ]:
mesh = Mesh("mesh2.xml")
plot(mesh, title="Finite element mesh")

> **TASK 1 guide**: As you change the parameters *hmin* and *hmax* above, remember to visualise again the mesh.



Each triangular cell that you see is an element. We can query the number of elements in the mesh.

In [ ]:
print(mesh.num_cells())

# Step 4. Define the elasticity problem

We will now setup the elasticity problem by finite element discretising the weak form that we have derived. First, we prescribe the applied stress. The applied stress is the total applied force ($10 \times 10^3$ N) divided by the cross-sectional area of the strip ($100$ mm $\times$ $1.7$ mm).

In [ ]:
applied_stress = 58             # applied normal stress (N/mm^2)

To specify the constitutive i.e. material behaviour, we prescribe the elastic constants i.e. Young's modulus $E$ and Poisson's ratio $ν$ which are given in the handout.

In [ ]:
E  = Constant(70000)            # Young's modulus (MPa i.e. N/mm^2)
nu = Constant(0.33)             # Poisson's ratio

We assume the plane stress condition. To this end, we define the corresponding 2D stress $\sigma$ and strain $ɛ$ components. Note that the 2D stress and strain tensor components are stored in matrices instead of vectors.

In [ ]:
def eps(v):
    return sym(grad(v))

In [ ]:
def sigma(v):
    return (E*nu/(1.0-nu*nu))*tr(eps(v))*Identity(2) + (E/(1.0+nu))*eps(v)

We now define the traction (i.e. natural or Neumann) boundary condition. Recall that the traction is the stress that we apply along the rightmost edge of the strip. We first obtain all boundary nodes of the mesh. We then mark if those nodes rest on the traction boundary (1 if true, 0 else). Finally, we define a boundary integral to compute the traction term.

In [ ]:
# return true if a vertex is on the right-hand side edge of the strip (x=L)
class IsTractionBoundary(SubDomain):
    def inside(self, x, on_boundary):
		    return near(x[0], L)
# get all boundary nodes and apply a scalar function over these vertices
mesh_boundary_bool = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
# initialise all boundary nodes as 0
mesh_boundary_bool.set_all(0)
# set boundary nodes that rest on the rightmost edge as 1
IsTractionBoundary().mark(mesh_boundary_bool, 1)
# we define a surface integral based on this function
ds = Measure("ds")(subdomain_data=mesh_boundary_bool)

We now proceed to implement the weak form in FEniCS. Recall that the weak form is given by

>$$ \int_\Omega (\mathbf{\nabla}_S \mathbf w)^T \mathbf{\sigma} (\mathbf u) \, d\Omega = \int_{\Gamma_t} \mathbf w^T \overline{\mathbf t} \, d \Gamma_t   $$

where $\Omega$ is the domain, $\Gamma_t$ is the traction boundary (in this case, the rightmost edge of the strip), $\mathbf{w}=\left[w_x\, w_y\right]^T$ is the vector of test functions, $\mathbf{\nabla}_S$ is the matrix of gradient operations, $ \mathbf{\sigma} = \left[\sigma_{xx}\, \sigma_{yy} \, \sigma_{xy} \right]^T$ is the stress vector and $ \overline{\mathbf t} = \left[\overline{ t}_x\,\,\overline{ t}_y \right]^T$ is the applied traction vector. Once again, note that in the current implementation, the stress and strain components are stored in matrices instead of vectors.

In [ ]:
s  = Constant((applied_stress, 0.0))                 # vector of applied stress
V  = VectorFunctionSpace(mesh, 'Lagrange', degree=2) # shape functions
u_ = TrialFunction(V)                                # trial function
w_ = TestFunction(V)                                 # test function
a  = inner(sigma(u_), eps(w_))*dx                    # internal virtual work
l  = inner(s, w_)*ds(1)                              # external virtual work

Next, we define the displacement (i.e. essential or Dirichlet) boundary conditions.

In [ ]:
# return true if the node is on the leftmost edge
def is_on_left(x, on_boundary):
    return near(x[0], 0.)

# return true if the node is on the bottom edge
def is_on_bottom(x, on_boundary):
    return near(x[1], 0.)

# return true if the node is on the rightmost edge
def is_on_right(x, on_boundary):
		return near(x[0], L)

# we define the relevant essential boundary condition on the bottom and left-hand edge of the domain
bcx = DirichletBC(V.sub(0), Constant((0.)), is_on_left)
bcy = DirichletBC(V.sub(1), Constant((0.)), is_on_bottom)
bc = [bcx, bcy]

> **TASK 2 guide**: If you prescribe a positive displacement on the rightmost edge instead of the prescribed traction, what do you need to change? Hints:
1.   Uncomment and experiment with the code below.
2.   Recall that on the same region, displacement and traction cannot be prescribed at the same time.


In [ ]:
# displacement = 0.5                  
# bc_displacement = DirichletBC(V.sub(0), Constant((displacement)), is_on_right)
# bc = [bcx, bcy, bc_displacement]                                     

# Step 5. Solve the linear system

We arrive at a linear system as a result of discretising the weak form. We solve the linear system using a linear solver provided by FEniCS.

In [ ]:
u = Function(V, name="Displacement")
solve(a == l, u, bc)

We examine qualitatively the finite element solution by visualising the displacement field.

In [ ]:
plot(u, mode="displacement")

Let us print the size of the displacement vector which is the number of degrees of freedom.

In [ ]:
u.vector()[:].size



> **TASK 3 guide** The higher the number of degrees of freedom, the more computationally expensive the linear system is. Can you state two factors that affect the number of degrees of freedom? Hints:
1.   Experiment with the number of elements.
2.   Experiment with the polynomial degree of the Lagrange shape functions.




#Step 6. Interpret analysis result

Finite element result is more than just colourful plots. We must interpret the results carefully. Here, we will use two Python modules namely [matplotlib](https://matplotlib.org/) for plotting and [numpy](https://numpy.org/) for linear algebra operations.

First, we evaluate the stress components near the vicinity of the hole.

In [ ]:
Vsig = TensorFunctionSpace(mesh, "DG", degree=1) # define finite element interpolation for the stress
sig = Function(Vsig, name="Stress")              # define stress function
sig.assign(project(sigma(u), Vsig))              # project sigma(u) onto the corresponding basis of shape functions
R = 14.8                                         # hole radius (mm)
print("Stress at (0,R):", sig(0.001, R+0.001))   # print the stress in the vicinity of the hole

Next, we plot the normal stress component $\sigma_{xx}$.

In [ ]:
from matplotlib import pyplot
stressplot = plot(sig[0, 0], wireframe=True, title="sigma_xx") 
pyplot.colorbar(stressplot)

> **TASK 4 guide**: What do you notice from the above stress field? Does the spatial variation of the stress depend on the magnitude of the applied traction? Hint: increase the applied traction and visualise again the stress field.

We now rescale the stress by the magnitude of the horizontal stress near the rightmost edge of the strip. Hence, we can estimate the stress concentration factor over the entire domain.

In [ ]:
sigma0 = sig(L, 0)[0]                               # define the horizontal stress near the rightmost edge of the strip
print(sigma0)                                       # check if this agrees with the traction boundary condition
sig_rescaled = Function(Vsig, name="Stress")
sig_rescaled.assign(project(sigma(u)/sigma0, Vsig)) # define the rescaled stress

We plot some profiles of the stress concentration factor in the domain.

> Do not use these plots directly in your report. The plots here are only for initial qualitative check. You should follow Step 7 below to export the results so that you can create plots for your report.




In [ ]:
# we import the numpy and math packages.
import numpy as np 
import math
# we define sets of horizontal and vertical coordinates
w = 50                              # plate half width (mm)
L = 150.                            # plate half length (mm)
n=math.floor(w/4)
x = np.linspace(R+0.01,L,n)
y = np.linspace(R+0.01,w,n)

In [ ]:
# we define horizontal and vertical rescaled stress profile along the leftmost and bottom edge of the domain
sig_xx_horizontalprofile = [sig_rescaled(xi, 0.)[0] for xi in x]
sig_xx_verticalprofile = [sig_rescaled(0., yi)[0] for yi in y]
sig_yy_horizontalprofile = [sig_rescaled(xi, 0.)[3] for xi in x]
sig_yy_verticalprofile = [sig_rescaled(0., yi)[3] for yi in y]

In [ ]:
# we plot the rescaled stress profiles along the bottom edge of the domain (y=0)
import matplotlib.pyplot as plt
plt.plot(x,sig_yy_horizontalprofile,'bo',label='sigma_yy/sigma0')
plt.plot(x,sig_xx_horizontalprofile,'ro',label='sigma_xx/sigma0')
plt.ylabel("rescaled stress")
plt.xlabel("distance center of the hole (mm)")
plt.title("rescaled stresses at y=0")
plt.legend()

In [ ]:
# we plot the rescaled stress profiles along the leftmost edge (x=0)
import matplotlib.pyplot as plt
plt.plot(y,sig_yy_verticalprofile,'bo',label='sigma_yy/sigma0')
plt.plot(y,sig_xx_verticalprofile,'ro',label='sigma_xx/sigma0')
plt.ylabel("rescaled stress")
plt.xlabel("distance center of the hole (mm)")
plt.title("rescaled stresses at x=0")
plt.legend()

In [ ]:
# we plot the rescaled stress along the rightmost edge and check if the value is equal to 1
bcsy = [sig_rescaled(L, yi)[0] for yi in y]
plt.plot(y,bcsy,'ro',label='sigma_xx/sigma0')
plt.ylabel("rescaled stress")
plt.xlabel("distance center of the plate (mm)")
plt.title("rescaled sigma_xx at x=L")
plt.ylim(0.5,1.5)
plt.legend()

# Step 7. Exporting result for post-processing

After examining qualitatively our numerical results, we need to quantify the accuracy of the finite element approximation. To this end, export your results so that you can post-process using a third-party software of your choice such as Matlab, Python and Excel.

## Step 7a. Mount your google drive and export results

To export the results, we first need to authorise your Cambridge Google Account. Make sure that you use the same Cambridge Google Account. Overall, the steps are:

1. Run the *code* cell below. A message will pop up.
2. Click **Connect to Google Drive**.
3. Select which Google account to proceed. Choose your Cambridge Google account.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Next, create a folder with the name **3D7** in your Google Drive. After that, run the following *code* cell to go to the folder directory.

In [ ]:
%cd /content/drive/My\ Drive/3D7

You can now save your rescaled stress profiles as .txt files. The block of code below will create two files. Each of these files has two rows. The first row contains the vector of $x$ coordinates in mm whereas the second contains the rescaled stress.

In [ ]:
import numpy as np
np.savetxt("stress-yy.txt", (x,sig_yy_horizontalprofile))
np.savetxt("stress-xx.txt", (x,sig_xx_horizontalprofile))

You can use the list command in Ubuntu shell to see if the files have been created.

In [ ]:
!ls

# Step 8. Additional post-processing methods

You are now able to post-process your results externally. The following provides additional guideline for post-processing your finite element result.

## Step 8a. Compute the norm of error

FEniCS has built-in functions to compute errors in different norms. Here, let us compare the finite element error between a coarse mesh and a refined mesh. More details can be found in the FEniCS article [Postprocessing computations](https://fenicsproject.org/pub/tutorial/html/._ftut1020.html) (see the section *Computing convergence rates*.)

> In order to execute the following cells, you must first save results using (i) a coarse mesh and (ii) a fine mesh. For each mesh, save the mesh file and the computed nodal displacements using the cell below. You should rename the files accordingly to align with the cell below.



In [ ]:
f_out = XDMFFile("u_coarse.xdmf") 
f_out.write_checkpoint(project(u, V), "Displacement", 0, XDMFFile.Encoding.HDF5, False) 
mesh_file = File("mesh_coarse.xml")
mesh_file << mesh

For each mesh, we will open the mesh file and the nodal displacements file.

In [ ]:
mesh = Mesh("mesh_coarse.xml")
V = VectorFunctionSpace(mesh, 'Lagrange', degree=2)
u_coarse = Function(V)
f_in =  XDMFFile("u_coarse.xdmf")
f_in.read_checkpoint(u_coarse,"Displacement",0)

In [ ]:
meshfine = Mesh("mesh_fine.xml")
Vfine = VectorFunctionSpace(meshfine, 'Lagrange', degree=2)
u_fine = Function(Vfine)
f_in =  XDMFFile("u_fine.xdmf")
f_in.read_checkpoint(u_fine,"Displacement",0)

Let us plot and compare the displacement field between the two meshes.

In [ ]:
plot(u_fine)

In [ ]:
plot(u_coarse)

Next, we extrapolate the coarse displacement on the fine mesh.

In [ ]:
u_coarse.set_allow_extrapolation(True)
u_interp_coarse = interpolate(u_coarse,Vfine)

We can visualise both displacement fields to examine the difference, if any.

In [ ]:
plot(inner(u_fine-u_interp_coarse,u_fine-u_interp_coarse))

Next, we compute the relative norm of error.

In [ ]:
u_interp_coarse.set_allow_extrapolation(True)
Error = errornorm(u_fine,u_interp_coarse)
Relativeerror = Error/norm(u_fine)

To determine the convergence rate, you can further refine the mesh and compute the finite element error. Ideally, the finite element error is computed using a known analytical solution. Since there is no analytical solution for the above problem, we use here the finite element solution obtained with a very fine mesh as the reference solution. In practice, you should, of course, assess the reference solution carefully if there is no analytical solution or experimental reference.

## Step 8b. Load and plot results

The following briefly demonstrates how to load the results from your Google Drive and compare against the analytical solution. Note that here we assume that the domain is infinite so that the analyical solution is known.

> You should not simply copy and paste the following plots for your report as the following is demonstrative only.

In [ ]:
# we define a fine set of horizontal coordinates
x_fine = np.linspace(R+0.01,L,1000)
# we define the analytical prediction for sigma_xx for the infinite plate case
stress_xx_theoretical=np.ones(x_fine.shape[0])-5*R*R/(2*np.power(x_fine,2))+3*np.power(R,4)/(2*np.power(x_fine,4))

In [ ]:
# we plot the analytical solution and the finite element simulation for the rescaled sigma_xx along the bottom boundary of the domain (y=0)
stress_xx = np.loadtxt("stress-xx.txt")
stress_yy = np.loadtxt("stress-yy.txt")
plt.plot(stress_xx[0],stress_xx[1],'bo',label="finite element result: sigma_xx/sigma0")
plt.plot(x_fine,stress_xx_theoretical,'r-',label="theoretical infinite plate prediction")
plt.ylabel("rescaled stress")
plt.xlabel("distance center of the hole (mm)")
plt.title("rescaled stress profile at y=0")
plt.legend()